# <span style="color:orange">**Inference notebook**</span>
- This notebook creates test predictions and generates a submission.parquet file.
- To execute it, you must generate the test CSV file with the 1st notebook (1_Feature_extraction.ipynb).
___

## **Librairies**

In [1]:
# Import useful librairies
from utils_librairies import *

## **Directory Paths**

In [2]:
# Import useful paths
from utils_paths import *

## **Utils**

In [3]:
# Import useful functions
from utils_models import *

___
___
# **Open files**

In [4]:
%%time

# Open test df
test = pd.read_csv("df_test.csv")

# Sort
test = test.sort_values(by = ['bldg_id']).reset_index(drop=True)

# Show
print(test.shape)
test.head(1)

(1440, 11021)
CPU times: user 6.73 s, sys: 413 ms, total: 7.15 s
Wall time: 7.23 s


,bldg_id,FFT_date_sum_15th_value,FFT_date_sum_2th_value,FFT_date_sum_5th_value,FFT_date_sum_9th_value,FFT_dayofweek_mean_2th_value,FFT_dayofweek_mean_3th_value,FFT_dayofweek_mean_4th_value,FFT_dayofweek_sum_3th_value,FFT_dayofweek_sum_4th_value,FFT_hour_mean_7th_value,FFT_hour_mean_8th_value,FFT_hour_minute_mean_3th_value,FFT_hour_minute_mean_4th_value,FFT_hour_minute_mean_5th_value,FFT_hour_minute_mean_6th_value,FFT_hour_minute_mean_9th_value,FFT_hour_minute_sum_10th_value,FFT_hour_minute_sum_12th_value,FFT_hour_minute_sum_15th_value,FFT_hour_minute_sum_4th_value,FFT_hour_sum_10th_value,FFT_hour_sum_11th_value,FFT_hour_sum_9th_value,FFT_week_mean_5th_value,FFT_week_sum_14th_value,FFT_week_sum_2th_value,FFT_week_sum_6th_value,FFT_week_sum_8th_value,MEAN_closing_quarter_estimation,MEAN_operating_hours_estimation_1,MEAN_operating_hours_estimation_MAX,MEDIAN_closing_quarter_estimation,MEDIAN_operating_hours_estimation_1,MEDIAN_operating_hours_estimation_MAX,STD_closing_quarter_estimation,STD_opening_quarter_estimation,STD_operating_hours_estimation,STD_operating_hours_estimation_1,STD_operating_hours_estimation_2,STD_operating_hours_estimation_3,STD_operating_hours_estimation_MAX,STD_operating_hours_estimation_MIN,autumn_weekend_ratio_is_equal_to_last_or_next_quarter,closing_diff_56th_quarter,closing_diff_56th_quarter_summer,closing_diff_56th_quarter_winter,closing_diff_57th_quarter,closing_diff_57th_quarter_all_year,closing_diff_57th_quarter_summer,...,10_45_energy_sum_diff_to_smooth,11_30_energy_sum_diff_to_smooth,06_00_abs_diff_sum,12_45_diff_sum,07_00_energy_max,05_45_energy_max,20_15_energy_max,21_15_energy_min,13_45_energy_max,06_00_energy_sum,21_00_energy_min,18_15_abs_diff_sum,23_45_energy_sum,16_45_energy_sum_diff_to_smooth,09_15_energy_sum,16_45_energy_max,19_00_diff_sum,08_30_energy_sum,19_45_energy_sum,19_45_energy_max,13_15_abs_diff_sum,15_15_diff_std,22_45_energy_min,00_45_energy_sum_diff_to_smooth,18_15_energy_max,19_00_energy_sum_diff_to_smooth,19_00_energy_min,19_00_energy_max,14_15_diff_sum,09_45_energy_min,06_45_energy_sum,05_45_energy_std,20_45_diff_sum,21_30_abs_diff_sum,07_00_energy_sum,05_30_energy_std,13_30_energy_sum_diff_to_smooth,13_15_energy_std,3th_inflexion_point_energy_std,06_45_energy_max,06_15_energy_max,15_00_energy_max,14_45_diff_std,14_30_diff_std,08_15_energy_sum,17_15_energy_std,15_00_diff_std,21_00_energy_std,14_15_energy_sum_diff_to_smooth,21_45_diff_sum
0,1,32.787938,478.380931,27.572189,152.900216,0.02589,0.0105,0.003272,36.46169,33.705276,0.058674,0.0387,1.353775,0.034507,0.515311,0.538501,0.1728,40.168412,89.619237,61.202093,12.594908,54.153985,76.656401,78.035685,0.044964,25.855414,443.014861,62.265882,28.934724,82.25,14.3125,16.3125,80.0,14.75,16.5,5.868347,7.614747,2.055433,2.419808,1.089725,0.811899,1.384907,2.05396,0.766857,0.051358,0.005098,-0.003048,0.008321,0.004608,-0.001118,...,-0.135649,-0.734323,0.459831,-0.986626,0.567679,0.641721,0.885567,1.767193,0.695031,0.152942,1.767193,0.515848,-1.543415,0.388018,1.578559,-0.527161,0.041625,0.742622,0.108072,0.402811,0.527313,-1.081141,1.138858,-0.302393,0.810538,-0.707843,2.081361,0.59927,0.597457,-1.060316,0.189427,0.39821,1.446603,0.677838,0.545522,0.387219,-0.171047,-0.185625,70,-0.860845,0.793755,0.800665,-0.430905,0.165992,1.732315,0.423095,0.825955,0.48392,0.51316,-0.65023


___
___
# **Inference**

### **Predict <i>building_stock_type</i>**

In [5]:
# -----------------------------------
# Prepare models paths and feats

# Model paths
model_paths = [model_dir+x for x in os.listdir(model_dir) if x.startswith('building_stock_type')]

# Features
feats = joblib.load(feats_dir + "feats_building_stock_type.joblib")

# -----------------------------------
# Iterate over models (several folds)
for i, path in enumerate(model_paths) :

    # Load model
    model = joblib.load(path)

    # Get predictions
    if i==0 :
        test_preds = model.predict_proba(test[feats]) / len(model_paths)
    else :
        test_preds += model.predict_proba(test[feats]) / len(model_paths)

# -----------------------------------
# Create predictions classes
test['building_stock_type'] = model.classes_[test_preds.argmax(axis=1)]

# Show
test['building_stock_type'].value_counts().to_frame()

,count
building_stock_type,
residential,780
commercial,660


### **Loop over residential and commercial columns**

In [6]:
# Columns to predict
cols_residential = ['in.bedrooms_res',
                    'in.cooling_setpoint_res',
                    'in.heating_setpoint_res',
                    'in.geometry_building_type_recs_res',
                    'in.geometry_floor_area_res',
                    'in.geometry_foundation_type_res',
                    'in.geometry_wall_type_res',
                    'in.heating_fuel_res',
                    'in.income_res',
                    'in.roof_material_res',
                    'in.tenure_res',
                    'in.vacancy_status_res',
                    'in.vintage_res']

cols_commercial = ['in.comstock_building_type_group_com',
                   'in.heating_fuel_com',
                   'in.hvac_category_com',
                   'in.number_of_stories_com',
                   'in.ownership_type_com',
                   'in.vintage_com',
                   'in.wall_construction_type_com',
                   'in.tstat_clg_sp_f..f_com',
                   'in.tstat_htg_sp_f..f_com',
                   'in.weekday_opening_time..hr_com',
                   'in.weekday_operating_hours..hr_com']

In [7]:
# Loop to predict one variable at a time
for var_to_predict in tqdm(cols_residential+cols_commercial) :
    
    # -----------------------------------
    # Create residential/commercial filter
    if var_to_predict.endswith('_res') :
        mask = (test["building_stock_type"] == 'residential')
    elif var_to_predict.endswith('_com') :
        mask = (test["building_stock_type"] == 'commercial')
    else :
        continue
    
    # -----------------------------------
    # Prepare models paths and feats
    
    # Model paths
    model_paths = [model_dir+x for x in os.listdir(model_dir) if x.startswith(var_to_predict)]
    
    # Features
    feats = joblib.load(feats_dir + f"feats_{var_to_predict}.joblib")
    
    # -----------------------------------
    # Iterate over models (several folds)
    for i, path in enumerate(model_paths) :
    
        # Load model
        model = joblib.load(path)

        # Get predictions
        if i==0 :
            test_preds = model.predict_proba(test.loc[mask, feats]) / len(model_paths)
        else :
            test_preds += model.predict_proba(test.loc[mask, feats]) / len(model_paths)
            
    # -----------------------------------
    # Select classes based on thresholds (not for 'building_stock_type' which has a perfect accuracy score)
    thresholds = joblib.load(thr_dir + f"thr_{var_to_predict}.joblib")   
    test.loc[mask, var_to_predict] = get_preds_from_thresholds(y_pred = test_preds,
                                                               labels=model.classes_,
                                                               thresholds=thresholds)
    

# Show
test[cols_residential+cols_commercial].head()

100%|██████████| 24/24 [00:31<00:00,  1.31s/it]


,in.bedrooms_res,in.cooling_setpoint_res,in.heating_setpoint_res,in.geometry_building_type_recs_res,in.geometry_floor_area_res,in.geometry_foundation_type_res,in.geometry_wall_type_res,in.heating_fuel_res,in.income_res,in.roof_material_res,in.tenure_res,in.vacancy_status_res,in.vintage_res,in.comstock_building_type_group_com,in.heating_fuel_com,in.hvac_category_com,in.number_of_stories_com,in.ownership_type_com,in.vintage_com,in.wall_construction_type_com,in.tstat_clg_sp_f..f_com,in.tstat_htg_sp_f..f_com,in.weekday_opening_time..hr_com,in.weekday_operating_hours..hr_com
0,1,75F,65F,Multi-Family with 5+ Units,500-749,Slab,Brick,Electricity,15000-19999,Composition Shingles,Renter,Occupied,2000s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,80F,65F,Single-Family Detached,2000-2499,Slab,Wood Frame,Natural Gas,200000+,"Tile, Clay or Ceramic",Owner,Occupied,2010s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Office,Electricity,Small Packaged Unit,2,owner_occupied,1980 to 1989,SteelFramed,74,69,8,8.75
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mercantile,NaturalGas,Small Packaged Unit,2,leased,Before 1946,SteelFramed,73,68,11,11.0
4,3,70F,68F,Single-Family Attached,1500-1999,Unheated Basement,Wood Frame,Natural Gas,15000-19999,Composition Shingles,Owner,Occupied,<1940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


___
___
# **Create submission file**

In [8]:
# Create submission from test df
sub = test[['bldg_id'] + cols_residential + cols_commercial]

# Set index
sub.index = sub['bldg_id']
sub = sub.drop(columns = ['bldg_id'])

# Delete '.0' suffix in string predictions ('10.0' -> '10'). This values appear due to the conversion "float -> str" by pandas
for k in sub.columns :
    sub[k] = sub[k].astype(str).apply(lambda x : x[:-2] if x.endswith('.0') else x)

# Export CSV
sub.to_parquet('submission.parquet')

# Show
print(sub.shape)
sub

(1440, 24)


,in.bedrooms_res,in.cooling_setpoint_res,in.heating_setpoint_res,in.geometry_building_type_recs_res,in.geometry_floor_area_res,in.geometry_foundation_type_res,in.geometry_wall_type_res,in.heating_fuel_res,in.income_res,in.roof_material_res,in.tenure_res,in.vacancy_status_res,in.vintage_res,in.comstock_building_type_group_com,in.heating_fuel_com,in.hvac_category_com,in.number_of_stories_com,in.ownership_type_com,in.vintage_com,in.wall_construction_type_com,in.tstat_clg_sp_f..f_com,in.tstat_htg_sp_f..f_com,in.weekday_opening_time..hr_com,in.weekday_operating_hours..hr_com
bldg_id,,,,,,,,,,,,,,,,,,,,,,,,
1,1,75F,65F,Multi-Family with 5+ Units,500-749,Slab,Brick,Electricity,15000-19999,Composition Shingles,Renter,Occupied,2000s,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,5,80F,65F,Single-Family Detached,2000-2499,Slab,Wood Frame,Natural Gas,200000+,"Tile, Clay or Ceramic",Owner,Occupied,2010s,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Office,Electricity,Small Packaged Unit,2,owner_occupied,1980 to 1989,SteelFramed,74,69,8,8.75
4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Mercantile,NaturalGas,Small Packaged Unit,2,leased,Before 1946,SteelFramed,73,68,11,11
5,3,70F,68F,Single-Family Attached,1500-1999,Unheated Basement,Wood Frame,Natural Gas,15000-19999,Composition Shingles,Owner,Occupied,<1940,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Warehouse and Storage,NaturalGas,Small Packaged Unit,1,leased,1946 to 1959,Mass,999,999,9,7.75
1437,4,70F,55F,Single-Family Detached,3000-3999,Unheated Basement,Wood Frame,Electricity,25000-29999,Composition Shingles,Owner,Vacant,1980s,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1438,2,72F,68F,Multi-Family with 2 - 4 Units,750-999,Unheated Basement,Wood Frame,Natural Gas,25000-29999,Composition Shingles,Owner,Occupied,<1940,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


___
___
# **Post-processing**

In [9]:
# Copy
original_sub = sub.copy()

# Accuracy on comstock_building_type_group_com is very high, so the predictions are most likely exact.
# -> In training data, buildings ('Healthcare', 'Warehouse and Storage', 'Lodging') have only one value for tstat_htg and tstat_clg : 999.
mask = (sub['in.comstock_building_type_group_com'].isin(['Healthcare', 'Warehouse and Storage', 'Lodging']))
sub.loc[mask, 'in.tstat_htg_sp_f..f_com'] = '999'
sub.loc[mask, 'in.tstat_clg_sp_f..f_com'] = '999'

# Accuracy on comstock_building_type_group_com is very high, so the predictions are most likely exact.
# -> In training data, buildings 'Education' have only one value for number_of_stories : 1.
mask = (sub['in.comstock_building_type_group_com'] == 'Education')
sub.loc[mask, 'in.number_of_stories_com'] = '1'

# Accuracy on comstock_building_type_group_com is very high, so the predictions are most likely exact.
# -> In training data, buildings ('Education', 'Food Service', 'Mercantile') have never the value 999 for tstat_htg and tstat_clg.
# So we replace 999 with most common value, and we clip values to avoid predicting never-seen or very-rare values.
mask = (sub['in.comstock_building_type_group_com'].isin(['Education', 'Food Service', 'Mercantile']))
sub.loc[mask, 'in.tstat_htg_sp_f..f_com'] = sub.loc[mask, 'in.tstat_htg_sp_f..f_com'].apply(lambda x : x if x!='999' else '68').clip('65', None)
sub.loc[mask, 'in.tstat_clg_sp_f..f_com'] = sub.loc[mask, 'in.tstat_clg_sp_f..f_com'].apply(lambda x : x if x!='999' else '72').clip('70', '75')

# Accuracy on in.vacancy_status_res is very high, so the predictions are most likely exact.
# -> In training data, vacancy_status_res = 'Vacant' implies heating_setpoint = '55F'.
mask = (sub['in.vacancy_status_res'] == 'Vacant')
sub.loc[mask, 'in.heating_setpoint_res'] = '55F'

# =====================================================================
# Print how many rows were post-processed
n = len(original_sub.compare(sub))
print(f"Rows changed : {n}/{len(sub)}.")

# =====================================================================
# Export CSV
sub.to_parquet('submission.parquet')

# Show
print(sub.shape)
sub.head()

Rows changed : 12/1440.
(1440, 24)


,in.bedrooms_res,in.cooling_setpoint_res,in.heating_setpoint_res,in.geometry_building_type_recs_res,in.geometry_floor_area_res,in.geometry_foundation_type_res,in.geometry_wall_type_res,in.heating_fuel_res,in.income_res,in.roof_material_res,in.tenure_res,in.vacancy_status_res,in.vintage_res,in.comstock_building_type_group_com,in.heating_fuel_com,in.hvac_category_com,in.number_of_stories_com,in.ownership_type_com,in.vintage_com,in.wall_construction_type_com,in.tstat_clg_sp_f..f_com,in.tstat_htg_sp_f..f_com,in.weekday_opening_time..hr_com,in.weekday_operating_hours..hr_com
bldg_id,,,,,,,,,,,,,,,,,,,,,,,,
1,1,75F,65F,Multi-Family with 5+ Units,500-749,Slab,Brick,Electricity,15000-19999,Composition Shingles,Renter,Occupied,2000s,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,5,80F,65F,Single-Family Detached,2000-2499,Slab,Wood Frame,Natural Gas,200000+,"Tile, Clay or Ceramic",Owner,Occupied,2010s,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Office,Electricity,Small Packaged Unit,2,owner_occupied,1980 to 1989,SteelFramed,74,69,8,8.75
4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Mercantile,NaturalGas,Small Packaged Unit,2,leased,Before 1946,SteelFramed,73,68,11,11
5,3,70F,68F,Single-Family Attached,1500-1999,Unheated Basement,Wood Frame,Natural Gas,15000-19999,Composition Shingles,Owner,Occupied,<1940,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
